# World Trade Data with Chain-like Analysis

I collect the data of world international trading.
And analysis them with the graph like method.
The results show the trading partnership in the world wide.

In [6]:
import numpy as np
import pandas as pd
from pathlib import Path
from tqdm.auto import tqdm

import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

## Data collection and processing

In [7]:
input_folder = Path('wits_en_trade_summary_allcountries_allyears')

records = []

# Collect
for csvFile in tqdm(input_folder.iterdir(), 'Iter csv files'):
    if not csvFile.is_file():
        continue

    df_raw = pd.read_csv(csvFile, encoding='gbk')

    df = df_raw[df_raw['Indicator'].map(lambda e: any([
        e == 'Trade (US$ Mil)-Top 5 Export Partner',
        e == 'Trade (US$ Mil)-Top 5 Import Partner'
    ]))]

    if len(df) == 0:
        continue

    df = df[['Reporter', 'Partner', 'Indicator Type', '2019']]
    df = df.dropna()
    df['fileName'] = csvFile.name

    records.append(df)

# Summary
df = pd.concat(records)

for col in ['Reporter', 'Partner', 'Indicator Type']:
    df[col] = df[col].map(lambda e: e.strip())

ignores = ['World', 'Unspecified']

for ign in ignores:
    df = df[df['Reporter'] != ign]
    df = df[df['Partner'] != ign]

full_df = df

full_df

Iter csv files: 0it [00:00, ?it/s]

,Reporter,Partner,Indicator Type,2019,fileName
10,Aruba,Cura鏰o,Export,1.54,en_ABW_AllYears_WITS_Trade_Summary.CSV
12,Aruba,Netherlands,Export,2.53,en_ABW_AllYears_WITS_Trade_Summary.CSV
13,Aruba,Netherlands,Import,96.99,en_ABW_AllYears_WITS_Trade_Summary.CSV
17,Aruba,Panama,Import,12.52,en_ABW_AllYears_WITS_Trade_Summary.CSV
19,Aruba,United Kingdom,Import,13.69,en_ABW_AllYears_WITS_Trade_Summary.CSV
...,...,...,...,...,...
22,Zimbabwe,Mozambique,Export,354.33,en_ZWE_AllYears_WITS_Trade_Summary.CSV
25,Zimbabwe,Singapore,Import,1208.72,en_ZWE_AllYears_WITS_Trade_Summary.CSV
26,Zimbabwe,South Africa,Export,2104.52,en_ZWE_AllYears_WITS_Trade_Summary.CSV
27,Zimbabwe,South Africa,Import,1852.44,en_ZWE_AllYears_WITS_Trade_Summary.CSV


In [8]:
export_df = full_df[full_df['Indicator Type'] == 'Export'].sort_values(by='2019', ascending=False)
export_df

,Reporter,Partner,Indicator Type,2019,fileName
17,European Union,United States,Export,494616.72,en_EUN_AllYears_WITS_Trade_Summary.CSV
17,China,United States,Export,418584.25,en_CHN_AllYears_WITS_Trade_Summary.CSV
27,Mexico,United States,Export,358660.83,en_MEX_AllYears_WITS_Trade_Summary.CSV
20,Canada,United States,Export,336214.85,en_CAN_AllYears_WITS_Trade_Summary.CSV
7,"Hong Kong, China",China,Export,296069.01,en_HKG_AllYears_WITS_Trade_Summary.CSV
...,...,...,...,...,...
13,Yemen,Eritrea,Export,0.21,en_YEM_AllYears_WITS_Trade_Summary.CSV
22,Sao Tome and Principe,Netherlands,Export,0.17,en_STP_AllYears_WITS_Trade_Summary.CSV
27,Sao Tome and Principe,Portugal,Export,0.17,en_STP_AllYears_WITS_Trade_Summary.CSV
29,Sao Tome and Principe,Slovenia,Export,0.16,en_STP_AllYears_WITS_Trade_Summary.CSV


In [9]:
import_df = full_df[full_df['Indicator Type'] == 'Import'].sort_values(by='2019', ascending=False)
import_df

,Reporter,Partner,Indicator Type,2019,fileName
10,United States,China,Import,472464.91,en_USA_AllYears_WITS_Trade_Summary.CSV
6,European Union,China,Import,470491.00,en_EUN_AllYears_WITS_Trade_Summary.CSV
17,United States,Mexico,Import,361320.94,en_USA_AllYears_WITS_Trade_Summary.CSV
18,European Union,United States,Import,327230.86,en_EUN_AllYears_WITS_Trade_Summary.CSV
8,United States,Canada,Import,326628.56,en_USA_AllYears_WITS_Trade_Summary.CSV
...,...,...,...,...,...
13,Sao Tome and Principe,China,Import,8.98,en_STP_AllYears_WITS_Trade_Summary.CSV
33,Comoros,Turkey,Import,8.80,en_COM_AllYears_WITS_Trade_Summary.CSV
16,St. Vincent and the Grenadines,Japan,Import,8.04,en_VCT_AllYears_WITS_Trade_Summary.CSV
24,Sao Tome and Principe,Nigeria,Import,6.77,en_STP_AllYears_WITS_Trade_Summary.CSV


## Local toolbox

In [10]:
def search_iter(df, rootReporter, results, level=0, maxLevel=3):
    '''
    Be ware it is an iterative function,
    make sure it ends properly.

    Search the df iteratively,
    the searching starts with the rootReporter,
    and stops with the maxLevel specified

    Args:
        :param: df: The raw dataframe to search in;
        :param: rootReporter: The reporter to be started with;
        :param: level: The current level;
        :param: maxLevel: The maximum level, default by 3;
        :param: results: A hash table containing the history step,
                         it is where the results are restored,
                         it also prevents the searching from being trapped in the loop.
    '''

    # maxLevel is reached
    if level > maxLevel:
        return

    # Select reporter
    select = df.query('Reporter == "{}"'.format(rootReporter))

    # Record partners
    furtherSearches = []
    for partner in select['Partner']:
        # Do nothing if exists
        if partner in results:
            continue

        # New partner found
        results[partner] = (level, results.get(
            rootReporter, (-1, []))[1] + [rootReporter])
        furtherSearches.append(partner)

    # Iterate searching
    for partner in furtherSearches:
        # Next searching
        search_iter(df, partner, results, level=level+1, maxLevel=maxLevel)

    return

def results_to_df(results, columns=dict()):
    '''
    Turn the results above into the dataframe,
    it is also sorted by level.

    Args:
        :param: results: The searching results;
        :param: columns: The columns to be added.

    Returns:
        :return: df: The sorted dataframe
    '''

    lst = []
    for key, value in results.items():
        lst.append([key, value[0], value[1]])

    df = pd.DataFrame(sorted(lst, key=lambda e: e[1]), columns=[
                      ['name', 'level', 'chain']])

    for key, value in columns.items():
        if key in df.columns:
            print('W: Can not add col: {}:{}'.format(key, value))
        df[key] = value

    return df


In [11]:

default_colormap = px.colors.sequential.Cividis_r

def mk_sankey_data(df, colormap=default_colormap):
    '''
    Prepare sankey data from df
    
    Args:
        :param: df: The input dataframe, it is generated by the results_to_df function in above;
        :param: colormap: The colormap, default by default_colormap.

    Returns:
        :return: label, color, name, target, value: The requirements of plotly Sankey graph, see https://plotly.com/javascript/sankey-diagram/#basic-sankey-diagram
        :return: root: The root country name.
    '''

    # Generate links
    links = dict()
    color_table = dict()

    root = df.iloc[0]['root']
    for i in range(len(df)):
        se = df.iloc[i]
        name = se['name']

        # if name == root:
        #     continue

        chain = [e for e in se['chain']]
        chain.append(name)

        level = se['level']
        color_table[name] = colormap[int(level) % len(colormap)]

        for i, n in enumerate(chain):
            if i == 0:
                continue

            m = chain[i-1]

            links[m] = links.get(m, dict())
            links[n] = links.get(n, dict())
            links[m][n] = links[m].get(n, 0)
            links[m][n] += 1

            
    # Generate label and color
    # Generate source, target and value

    label = [k for k in links]
    color = [e for e in label]

    for i, lb in enumerate(label):
        if lb == root:
            color[i] = colormap[0]
            continue
        color[i] = color_table[lb]

    source = []
    target = []
    value = []

    for src, a in links.items():
        for tar, val in a.items():
            source.append(label.index(src))
            target.append(label.index(tar))
            value.append(val)

    return label, color, source, target, value, root


# Display the sankey data
def mk_sankey_fig(label, color, source, target, value, title):
    '''
    Make sankey figure from inputs

    Args:
        :param: label, color, source, target, value: The sankey data generated by the mk_sankey_data;
        :param: title: The figure title

    Return:
        :return: fig: The figure.
    '''
    fig = go.Figure(data=[go.Sankey(
        node = dict(
        pad = 15,
        thickness = 20,
        line = dict(color = "black", width = 0.5),
        label = label,
        color = color,
        ),
        link = dict(
        source = source,
        target = target,
        value = value
    ))])

    fig.update_layout(title_text=title, font_size=10)

    return fig


## Display sankey graph for selected country

In [12]:
''' 
It is C styled programming,
make the space in results, and fill it with iterative searching,
and convert the results into sorted dataframe
'''

# ---------------------------------------
# Input
country = 'China'
trade = 'Export'
inp_df = export_df 

# ---------------------------------------
# Workload
results = dict()

search_iter(inp_df, country, results, level=1, maxLevel=100)

trace_df = results_to_df(results, dict(root=country, trade=trade))

label, color, source, target, value, root = mk_sankey_data(trace_df)

fig = mk_sankey_fig(label, color, source, target, value, '{}, ({})'.format(country, trade))

# ---------------------------------------
# Output
fig.show()
trace_df

,name,level,chain,root,trade
0,United States,1,[China],China,Export
1,"Hong Kong, China",1,[China],China,Export
2,Japan,1,[China],China,Export
3,"Korea, Rep.",1,[China],China,Export
4,Vietnam,1,[China],China,Export
5,Canada,2,"[China, United States]",China,Export
6,Mexico,2,"[China, United States]",China,Export
7,China,2,"[China, United States]",China,Export
8,United Kingdom,2,"[China, United States]",China,Export
9,India,2,"[China, Hong Kong, China]",China,Export


## Complex analysis for global partnership

In [51]:


inp_df_dict = dict(
    Import=import_df,
    Export=export_df
)

trade = 'Import'

degree_df_list = []
matrix_dict = dict()

for trade in inp_df_dict:
    # Load countries, the matrix, and the matrix_table
    inp_df = inp_df_dict[trade]
    countries = sorted(set(inp_df['Reporter'].values))

    n = len(countries)
    matrix = np.zeros((n, n))
    matrix_idx_table = dict()
    for i, n in enumerate(countries):
        matrix_idx_table[n] = i

    for country in tqdm(countries, 'Countries ({})'.format(trade)):
        results = dict()

        search_iter(inp_df, country, results, level=1, maxLevel=100)

        trace_df = results_to_df(results, dict(root=country, trade=trade))

        label, color, source, target, value, root = mk_sankey_data(trace_df)

        for src, tar, val in zip(source, target, value):
            try:
                i = matrix_idx_table[label[src]]
                j = matrix_idx_table[label[tar]]
            except KeyError:
                continue

            matrix[i, j] += val

    matrix_dict[trade] = (matrix, matrix_idx_table)

    ''' 
    degree1 is the degree of directing to the other countries;
    degree2 is the degree of being directed to the other countries;
    '''

    degree_df = pd.DataFrame()
    degree_df['country'] = countries
    degree_df['degree1'] = np.sum(matrix, axis=0)
    degree_df['degree2'] = np.sum(matrix, axis=1)
    degree_df['trade'] = trade

    degree_df_list.append(degree_df)


degree_df = pd.concat(degree_df_list, axis=0) 
degree_df['degree'] = degree_df['degree1'] + degree_df['degree2']
degree_df

Countries (Import):   0%|          | 0/131 [00:00<?, ?it/s]

Countries (Export):   0%|          | 0/131 [00:00<?, ?it/s]

,country,degree1,degree2,trade,degree
0,Afghanistan,0.0,30.0,Import,30.0
1,Antigua and Barbuda,0.0,28.0,Import,28.0
2,Argentina,30.0,51.0,Import,81.0
3,Armenia,0.0,29.0,Import,29.0
4,Aruba,0.0,28.0,Import,28.0
...,...,...,...,...,...
126,Uruguay,0.0,28.0,Export,28.0
127,Vietnam,131.0,25.0,Export,156.0
128,Yemen,0.0,27.0,Export,27.0
129,Zambia,0.0,26.0,Export,26.0


### Display global partnerships

In [18]:
trades = [e for e in inp_df_dict]
fig = make_subplots(rows=1, cols=2, subplot_titles=trades)

for j, trade in enumerate(trades):
    mat = matrix_dict[trade][0]
    names = [e for e in matrix_dict[trade][1]]
    f = px.imshow(np.log10(mat), x=names, y=names, title='{} (log10)'.format(trade))
    fig.add_trace(trace=f.data[0], row=1, col=j+1)

fig.show()

C:\Users\zcc\AppData\Local\Temp\ipykernel_5368\4168598956.py:7: RuntimeWarning:

divide by zero encountered in log10



### Display degrees of being the partners 

In [61]:

df_raw = degree_df.query('degree1 > 100')

# Display 1
df = df_raw.sort_values(by='degree1', ascending=False)
f = px.bar(df, x='country', y='degree1', color='country', pattern_shape='trade', barmode='stack', title='Degree')
f.show()

# Display 2
df = df_raw.sort_values(by='degree1', ascending=False)
f = px.scatter(df, x='degree1', y='degree2', color='country', symbol='trade', title='Degree Pair')
f.show()


In [14]:
for trade in trades:
    # Display 1
    df_raw = degree_df.query('trade == "{}"'.format(trade)).query('degree1 > 100')
    df = df_raw.sort_values(by='degree1', ascending=False)
    px.bar(df, x='country', y='degree1', color='country', title='Degree in {}'.format(trade)).show()

    # Display 2
    df = df_raw.sort_values(by='degree1', ascending=False)
    px.scatter(df, x='degree1', y='degree2', color='country', title='Degree Pair in {}'.format(trade)).show()